## Oracle Network 

In [1]:
import pandas as pd
import numpy as np

import torch

#imports necessary to define a neural network 
import torch.nn as nn
import torch.nn.functional as F

#ensure you are using GPU.
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

dtype = torch.cuda.FloatTensor

cuda:0


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

trdt1=pd.read_csv('C:\\Users\\Ant Pc\\GitHub\\pROJ\\Genotype\\TrainingGenotype.csv')
trdt=pd.read_csv('C:\\Users\\Ant Pc\\GitHub\\pROJ\\Geneexpression0\\traingeneexpression.csv',header=None)
tstdt=pd.read_csv('C:\\Users\\Ant Pc\\GitHub\\pROJ\\Geneexpression0\\testgeneexpression.csv',header=None)

tstdt=np.array(tstdt)
tstdt=np.transpose(tstdt)
tstdt=pd.DataFrame(tstdt)

trdt=np.array(trdt)
trdt=np.transpose(trdt)
trdt=pd.DataFrame(trdt)

results1=trdt1['Phenotype1']
results2=trdt1['Phenotype2']
# convert matrices to pytorch tensors on gpu

trdttensor=torch.from_numpy(trdt.values).type(dtype)
tstdttensor=torch.from_numpy(tstdt.values).type(dtype)
results1tensor=torch.from_numpy(results1.values).type(dtype)
results2tensor=torch.from_numpy(results2.values).type(dtype)

print(trdttensor.shape)
print(tstdttensor.shape)
print(results2tensor.shape)
print(results1tensor.shape)

torch.Size([200, 28395])
torch.Size([30, 28395])
torch.Size([200])
torch.Size([200])


In [3]:
gold=pd.read_csv('C:/Users/Ant Pc/GitHub/PROJECT/DATA/DREAM5_SysGenB2_GoldStandard.csv', sep='\t')
gold.drop(gold.columns[-1],axis=1,inplace=True)

gold2=gold.iloc[1,:]
gold2=gold2.values

dt2=np.transpose(gold2)

gold1=gold.iloc[0,:]
gold1=gold1.values

dt1=np.transpose(gold1)

In [4]:
randmatr=pd.read_csv('C:/Users/Ant Pc/GitHub/PROJECT/Randmatr.csv', header=None).values
class Net1(nn.Module):

    def __init__(self):
        super(Net1, self).__init__()
        
        # an affine operation: y = Wx + b
        
        self.rand=nn.Linear(28395,1000).cuda()
        self.fc1=nn.Linear(1000,3200).cuda()
        self.fc2=nn.Linear(3200,3200).cuda()
        self.fc3=nn.Linear(3200,3200).cuda()
        self.fc6 = nn.Linear(3200,1).cuda()
        
        #set weights as random projection matrix
        self.rand.weight.data = torch.transpose(torch.from_numpy(randmatr),0,1).type(dtype)
        
    def forward(self, x):
        x = self.rand(x).cuda()
        x = F.relu(x).cuda()
        x = F.relu(self.fc1(x)).cuda() 
        x = F.relu(self.fc2(x)).cuda()   
        x = F.relu(self.fc3(x)).cuda()      
        x = self.fc6(x).cuda()
        
        return x

model = Net1()

#use gpu for all computations in model
model.cuda()
print(model)

Net1(
  (rand): Linear(in_features=28395, out_features=1000, bias=True)
  (fc1): Linear(in_features=1000, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=3200, bias=True)
  (fc3): Linear(in_features=3200, out_features=3200, bias=True)
  (fc6): Linear(in_features=3200, out_features=1, bias=True)
)


In [5]:
#trainingloop
import torch.optim as optim

lr = 0.001
batchsize=20

batches=len(trdttensor)/batchsize

epochs=50

# create your optimizer
optimizer = optim.Adagrad(model.parameters(), lr)


for i in range(epochs):
    
    for j in range(int(batches)):
        
        #forward pass
        out=model(trdttensor[j:j+batchsize,:].type(dtype))

        #compute loss
        criterion = nn.MSELoss()
        loss = criterion(torch.reshape(out,[20]).type(dtype),torch.reshape(results2tensor[j:j+batchsize],[20]).type(dtype)).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        #use optimiser to update
        optimizer.step()
    c=nn.L1Loss()
    print(c(torch.reshape(model(tstdttensor),[30]),torch.from_numpy(np.array(dt2)).type(dtype)),i)

tensor(16.2362, device='cuda:0', grad_fn=<L1LossBackward>) 0
tensor(6.6995, device='cuda:0', grad_fn=<L1LossBackward>) 1
tensor(3.8907, device='cuda:0', grad_fn=<L1LossBackward>) 2
tensor(2.5916, device='cuda:0', grad_fn=<L1LossBackward>) 3
tensor(2.5141, device='cuda:0', grad_fn=<L1LossBackward>) 4
tensor(2.7419, device='cuda:0', grad_fn=<L1LossBackward>) 5
tensor(3.1076, device='cuda:0', grad_fn=<L1LossBackward>) 6
tensor(5.6500, device='cuda:0', grad_fn=<L1LossBackward>) 7
tensor(5.6306, device='cuda:0', grad_fn=<L1LossBackward>) 8
tensor(5.8528, device='cuda:0', grad_fn=<L1LossBackward>) 9
tensor(5.9091, device='cuda:0', grad_fn=<L1LossBackward>) 10
tensor(5.9468, device='cuda:0', grad_fn=<L1LossBackward>) 11
tensor(5.9385, device='cuda:0', grad_fn=<L1LossBackward>) 12
tensor(5.8785, device='cuda:0', grad_fn=<L1LossBackward>) 13
tensor(5.7904, device='cuda:0', grad_fn=<L1LossBackward>) 14
tensor(5.6993, device='cuda:0', grad_fn=<L1LossBackward>) 15
tensor(5.6115, device='cuda:0', g

In [6]:
model(trdttensor[0:20,:])

tensor([[19.3808],
        [23.3114],
        [20.4759],
        [21.9783],
        [18.6246],
        [20.9809],
        [20.5792],
        [19.0692],
        [21.4399],
        [20.6262],
        [20.0444],
        [19.0925],
        [22.4191],
        [27.2825],
        [18.9363],
        [21.3597],
        [21.1166],
        [18.7511],
        [21.1818],
        [18.5806]], device='cuda:0', grad_fn=<AddmmBackward>)

## OGGN

In [7]:
#random input matrix
nsamples=3000
X=np.random.randn(nsamples,6)

Xtensor=torch.from_numpy(X).type(dtype)
print(Xtensor[:,0:30])

tensor([[ 0.1122, -0.8336, -0.8104, -1.0369, -0.6032,  0.8279],
        [-0.0751, -0.8436, -0.4878,  0.2767, -1.7381,  1.8533],
        [ 0.2180, -0.3195,  1.4491, -0.3874, -0.7247,  0.3371],
        ...,
        [-0.1126,  0.5114, -0.2978, -0.3862,  0.0341, -1.5288],
        [-0.2055,  2.4584,  1.6704,  1.4604,  1.7337, -1.9728],
        [ 2.0211, -0.1911,  0.4957, -0.5623, -1.5291,  0.1290]],
       device='cuda:0')


In [8]:
def customloss(matr):
    #input is tensor of size(1,28395)
    
    inpu=matr
    
    criterion = nn.MSELoss()
    
    pred=model(matr).type(dtype)
    ideal=5*torch.ones_like(pred).type(dtype)
    loss=criterion(pred,ideal).type(dtype)
    
    return loss

In [9]:
#Neural network to generate geneexpression values
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        # an affine operation: y = Wx + b
        self.inp =nn.Linear(6,3200).cuda()
        self.fc1=nn.Linear(3200,3200).cuda()
        self.fc2 = nn.Linear(3200,28395).cuda()
        
    def forward(self, x):
        x=F.relu(self.inp(x)).cuda()
        x = self.fc1(x).cuda()
        x = self.fc2(x).cuda()
        x=torch.abs(x).cuda()
        x[x>10]=x[x>10]/10
        x[x<1]=x[x<1]*100
        x=x.cuda()
        return x

net = Net()
net.cuda()
print(net)

Net(
  (inp): Linear(in_features=6, out_features=3200, bias=True)
  (fc1): Linear(in_features=3200, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=28395, bias=True)
)


In [10]:
#trainingloop
import torch.optim as optim

lr = 0.001
batchsize=100

batches=nsamples/batchsize

epochs=20

# create your optimizer
optimizer = optim.Adagrad(net.parameters(), lr)

for i in range(epochs):
    
    for j in range(int(batches)):
        
        #forward pass
        out=net(Xtensor[j:j+batchsize,:]).type(dtype)

        #compute loss
        loss = customloss(out).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        optimizer.step()
    print(loss,i)

tensor(20977.6543, device='cuda:0', grad_fn=<MseLossBackward>) 0
tensor(53.8950, device='cuda:0', grad_fn=<MseLossBackward>) 1
tensor(685.9599, device='cuda:0', grad_fn=<MseLossBackward>) 2
tensor(3608.3394, device='cuda:0', grad_fn=<MseLossBackward>) 3
tensor(45.3038, device='cuda:0', grad_fn=<MseLossBackward>) 4
tensor(9.6268, device='cuda:0', grad_fn=<MseLossBackward>) 5
tensor(6.3959, device='cuda:0', grad_fn=<MseLossBackward>) 6
tensor(189.8401, device='cuda:0', grad_fn=<MseLossBackward>) 7
tensor(143.8048, device='cuda:0', grad_fn=<MseLossBackward>) 8
tensor(7.5007, device='cuda:0', grad_fn=<MseLossBackward>) 9
tensor(7.4925, device='cuda:0', grad_fn=<MseLossBackward>) 10
tensor(3.3244, device='cuda:0', grad_fn=<MseLossBackward>) 11
tensor(4.2360, device='cuda:0', grad_fn=<MseLossBackward>) 12
tensor(2.3196, device='cuda:0', grad_fn=<MseLossBackward>) 13
tensor(2.8710, device='cuda:0', grad_fn=<MseLossBackward>) 14
tensor(2.6677, device='cuda:0', grad_fn=<MseLossBackward>) 15
ten

In [11]:
net(Xtensor[0:4,0:10])

tensor([[15.1280, 14.5481, 17.3364,  ..., 17.1182,  4.1342, 39.2478],
        [13.0939,  0.7497,  4.8362,  ..., 14.2533, 11.8087, 19.2751],
        [ 1.4948,  4.3236,  7.4085,  ..., 11.1555,  5.1018,  8.9809],
        [23.4049, 16.7881,  3.2906,  ...,  5.9219,  2.5359,  1.1345]],
       device='cuda:0', grad_fn=<IndexPutBackward>)

In [12]:
customloss(net(Xtensor[4,:]))

tensor(1.7759, device='cuda:0', grad_fn=<MseLossBackward>)

In [13]:
model(net(Xtensor[0:20,:]))

tensor([[2.2062],
        [6.9819],
        [4.1511],
        [5.6062],
        [3.6674],
        [8.0124],
        [3.7984],
        [5.7121],
        [3.5844],
        [4.5113],
        [5.7482],
        [4.3238],
        [2.4769],
        [4.6265],
        [6.9008],
        [4.1227],
        [3.1758],
        [4.5578],
        [4.6578],
        [5.9837]], device='cuda:0', grad_fn=<AddmmBackward>)

In [ ]:
preddata=net(Xtensor).detach().cpu().numpy()
preddata=pd.DataFrame(preddata)
preddata=preddata.mean()
preddata.to_csv('d0.csv')
print((preddata.mean()))